In [51]:
import cv2
from cv2 import aruco
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import matplotlib.pyplot as plt

In [52]:
_pth = os.path.dirname(os.getcwd())
_parent_folder = "data"
_calib_folder_name = "calib_1200_400_af"

_webcam_calib_folder = os.path.join(
    _pth, _parent_folder, "calibration", _calib_folder_name
)
_webcam_calib_video = os.path.join(_webcam_calib_folder, "webcam_color.msgpack")
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join(_webcam_calib_folder, "webcam_calibration.msgpack")

In [53]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)
markerLength = 0.05
markerSeperation = 0.01

board = aruco.GridBoard(
    size=[1, 1],
    markerLength=markerLength,
    markerSeparation=markerSeperation,
    dictionary=ARUCO_DICT,
)
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)
_video_length = 0

for _frame in _video_data:
    _video_length += 1

_video_file.close()

In [54]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

marker_corners = []
marker_ids = []
counter = 0
rnd = np.random.choice(_video_length, 150, replace=False)
for idx, data_points in enumerate(_video_data):
    corners, ids = data_points

    marker_corners.append(corners)
    marker_ids.append(ids)
    counter += 1

_video_file.close()

In [55]:
np.zeros((4, 1, 3)).shape
rnd = np.random.choice(_video_length, 50, replace=False)

(4, 1, 3)

In [57]:
processed_image_points = []
processed_object_points = []
for _f in range(len(marker_corners)):
    if _f in rnd:
        current_object_points, current_image_points = board.matchImagePoints(
            marker_corners[_f], marker_ids[_f]
        )
        try:
            if current_object_points.any() and current_image_points.any():
                if current_object_points.shape == np.zeros((4, 1, 3)).shape:
                    new_matrix = np.zeros((4, 1, 3), dtype=np.float32)
                    new_matrix[:, :, :2] = current_image_points
                    processed_image_points.append(new_matrix)
                    processed_object_points.append(current_object_points)
        except:
            pass

In [58]:
useDeng = False
cameraNumber = 4
useFisheye = True

patternSize = (6,4)
squareSize = 50
imgSize = (1200,480)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize,squareSize)
worldPoints = []
imagePoints = [] 

In [59]:
boardPoints.shape

(24, 3)

In [60]:
def undistortFisheye(imgpath,K,D,DIM,axis,method,scale,corners):
    img = cv2.imread(imgpath)
    '''
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, DIM, np.eye(3), balance=0)
    new_K = K.copy()
    new_K[0,0]=K[0,0]*scale
    new_K[1,1]=K[1,1]*scale
    '''
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    fx = K[0,0]
    fy = K[1,1]
    cx = K[0,2]
    cy = K[1,2]
    undCorners_norm = cv2.fisheye.undistortPoints(corners, K, D)
    undCorners_norm = undCorners_norm.reshape(-1,2)
    undistCorners = np.zeros_like(undCorners_norm)
    for i, (x, y) in enumerate(undCorners_norm):
        px = x*fx + cx
        py = y*fy + cy
        undistCorners[i,0] = px
        undistCorners[i,1] = py    
    cv2.drawChessboardCorners(undistorted_img, patternSize, undistCorners, _)

    axs[axis].imshow(undistorted_img[:,:,::-1])
    axs[axis].axis('off')
    axs[axis].set_title('undistort '+method)
    #cv2.imwrite('undistort'+method+'.png', undistorted_img)

    return corners,undistCorners

def undistortPinhole(imgpath,K,D,DIM,axis,method,corners):
    img = cv2.imread(imgpath)
    
    new_K, roi = cv2.getOptimalNewCameraMatrix(K, D, DIM, 1, DIM)
    undistorted_img = cv2.undistort(img,K,D, None, new_K)  

    undCorners_norm = cv2.undistortPoints(corners, K, D).reshape(-1,2)
    # remove normalization
    fx = new_K[0,0]
    fy = new_K[1,1]
    cx = new_K[0,2]
    cy = new_K[1,2]
    undistCorners = np.zeros_like(undCorners_norm)
    for i, (x, y) in enumerate(undCorners_norm):
        px = x*fx + cx
        py = y*fy + cy
        undistCorners[i,0] = px
        undistCorners[i,1] = py    
    cv2.drawChessboardCorners(undistorted_img, patternSize, undistCorners, _)
    axs[axis].imshow(undistorted_img[:,:,::-1])
    axs[axis].axis('off')
    axs[axis].set_title('undistorted '+method)   


In [61]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

marker_corners = []
marker_ids = []
counter = 0
rnd = np.random.choice(_video_length, 150, replace=False)
for idx, data_points in enumerate(_video_data):
    corners, ids = data_points
    
    if corners is not None:
        
        if len(corners) == 12:
        # cornersRefined = cv2.cornerSubPix(gray, corners, (7,7), (-1,-1), criteria)
            imagePoints.append(corners)
            worldPoints.append(boardPoints)

_video_file.close()

In [64]:
if useFisheye:
    flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
    calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)
    ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), np.expand_dims(np.asarray(imagePoints), -2), imgSize, None, None,
                                                                flags=flagsCalib,criteria=calibrateCriteria)
else:
    flagsCalib = cv2.CALIB_RATIONAL_MODEL
    ret, cameraMatrix, k, rvecs, tvecs = cv2.calibrateCamera(worldPoints, imagePoints, imgSize, None, None,
                                                               flags=flagsCalib)

# print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", k)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fisheye.cpp:754: error: (-215:Assertion failed) imagePoints.type() == CV_32FC2 || imagePoints.type() == CV_64FC2 in function 'cv::fisheye::calibrate'
